# Subtitle Generator using OpenAI Whisper

මෙම Notebook သည် video သို့မဟုတ် audio file များမှ စာတန်းထိုး (.srt) များကို အလိုအလျောက် ဖန်တီးပေးပါသည်။ 

**အရေးကြီး:** လုပ်ငန်းမစတင်မီ Google Colab ၏ Runtime ကို GPU သို့ပြောင်းရန် မမေ့ပါနှင့်။
*(Menu -> Runtime -> Change runtime type -> Hardware accelerator -> GPU -> Save)*

In [ ]:
#@title 1. Install Libraries
#@markdown Google Colab ရဲ့ Runtime ကို GPU သို့ပြောင်းပါ။
#@markdown (Runtime -> Change runtime type -> Hardware accelerator -> GPU)

# GPU ချိတ်ဆက်ထားခြင်းရှိမရှိ စစ်ဆေးခြင်း
!nvidia-smi

# Video/Audio file တွေအတွက် မရှိမဖြစ်လိုအပ်တဲ့ ffmpeg ကို install လုပ်ခြင်း
!apt update && apt install -y ffmpeg

# OpenAI Whisper library ၏ နောက်ဆုံး version ကို install လုပ်ခြင်း
!pip install -U openai-whisper

## 2. Mount Google Drive

In [ ]:
#@title Mount Google Drive
#@markdown Google Drive ကိုချိတ်ဆက်ပြီး အသုံးပြုခွင့် (permission) ပေးပါ။

from google.colab import drive
drive.mount('/content/drive')

## 3. Transcribe and Generate Subtitles

In [ ]:
#@title Transcribe and Generate Subtitles

import os
import whisper
from datetime import timedelta
import subprocess

#@markdown ### 📁 Input and Output Settings
#@markdown **Path_of_Video_or_Audio:** Drive ထဲက video/audio file ပေါ် right-click နှိပ်ပြီး "Copy path" ကိုရွေးကာ ဒီနေရာမှာ paste ချပါ။
Path_of_Video_or_Audio = "/content/drive/MyDrive/video.mp4" #@param {type:"string"}
#@markdown **Output_Directory:** စာတန်းထိုးဖိုင်များသိမ်းဆည်းမည့် Folder အမည်။ Colab ရဲ့ လက်ရှိ directory မှာ အလိုအလျောက် တည်ဆောက်ပါမည်။
Output_Directory = "results" #@param {type:"string"}

#@markdown ---
#@markdown ### ⚙️ Transcription Settings
#@markdown **Model_Size:** Model အရွယ်အစားကြီးလေ ပိုတိကျလေ၊ အချိန်ပိုကြာလေ ဖြစ်ပါတယ်။ (e.g., 'small', 'medium', 'large-v2')
Model_Size = "small" #@param ['tiny', 'base', 'small', 'medium', 'large-v1', 'large-v2']
#@markdown **Language:** Video/Audio ရဲ့ မူရင်းဘာသာစကား code ကိုထည့်ပါ။ (e.g., `en` for English, `ja` for Japanese, `my` for Burmese)။ `None` ထားပါက အလိုအလျောက်စစ်ဆေးပါမည်။
Language = "ja" #@param {type:"string"}
#@markdown **Task:** `transcribe` (မူရင်းဘာသာဖြင့်စာอด), `translate` (အင်္ဂလိပ်ဘာသာသို့ပြန်ဆို)
Task = "translate" #@param ["transcribe", "translate"]
#@markdown **Subtitle_Action:**
#@markdown - `None`: .srt စာတန်းထိုးဖိုင်သာ ထုတ်ပေးမည်။
#@markdown - `burn`: စာတန်းထိုးကို video ပေါ်တွင် အသေကပ်ပြီး video file အသစ်ထုတ်ပေးမည် (Hardsub)။
#@markdown - `add`: စာတန်းထိုးကို video file ထဲသို့ track တစ်ခုအနေနဲ့ ထည့်ပေးမည် (Softsub)။
Subtitle_Action = "None" #@param ["None", "add", "burn"]

# --- Main Logic ---

# 1. Output directory ကို ဖန်တီးခြင်း
if not os.path.exists(Output_Directory):
    os.makedirs(Output_Directory)

# 2. Whisper model ကို သတ်မှတ်ထားသောအရွယ်အစားဖြင့် load လုပ်ပါ
print(f"Loading Whisper model: '{Model_Size}'...")
model = whisper.load_model(Model_Size)
print("Model loaded successfully.")

# 3. Transcription ပြုလုပ်ခြင်း
print(f"Starting {Task} for: {Path_of_Video_or_Audio}")
result = model.transcribe(
    Path_of_Video_or_Audio,
    language=(Language if Language else None),
    task=Task,
    fp16=False # If you have a problem, change to False
)
print("Transcription completed.")

# 4. SRT file format ပြောင်းလဲပြီး သိမ်းဆည်းခြင်း
def format_timestamp(seconds):
    """Seconds ကို SRT timestamp format (HH:MM:SS,ms) သို့ပြောင်းပေးသည်။"""
    delta = timedelta(seconds=seconds)
    hours, remainder = divmod(delta.seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    milliseconds = delta.microseconds // 1000
    return f"{hours:02}:{minutes:02}:{seconds:02},{milliseconds:03}"

base_filename = os.path.splitext(os.path.basename(Path_of_Video_or_Audio))[0]
srt_filename = os.path.join(Output_Directory, f"{base_filename}.srt")

with open(srt_filename, "w", encoding="utf-8") as srt_file:
    for i, segment in enumerate(result["segments"]):
        start_time = format_timestamp(segment['start'])
        end_time = format_timestamp(segment['end'])
        text = segment['text'].strip()
        srt_file.write(f"{i + 1}\n")
        srt_file.write(f"{start_time} --> {end_time}\n")
        srt_file.write(f"{text}\n\n")

print(f"SRT file saved to: {srt_filename}")

# 5. Subtitle Action (burn or add)
if Subtitle_Action != "None":
    output_video_path = os.path.join(Output_Directory, f"{base_filename}_{Subtitle_Action}.mp4")
    print(f"Performing '{Subtitle_Action}' action... Output will be saved to {output_video_path}")

    if Subtitle_Action == "burn":
        # Hardsub: စာတန်းထိုးကို video ပေါ်မှာ အသေကပ်ခြင်း
        command = [
            'ffmpeg', '-y', '-i', Path_of_Video_or_Audio, '-vf',
            f"subtitles='{srt_filename}'", '-c:a', 'copy',
            output_video_path
        ]
    elif Subtitle_Action == "add":
        # Softsub: စာတန်းထိုးကို track အနေနဲ့ ထည့်ခြင်း
        command = [
            'ffmpeg', '-y', '-i', Path_of_Video_or_Audio, '-i', srt_filename,
            '-c', 'copy', '-c:s', 'mov_text',
            '-metadata:s:s:0', 'language=eng', # Subtitle language code
            output_video_path
        ]

    # Run the ffmpeg command
    try:
        process = subprocess.run(command, check=True, capture_output=True, text=True)
        print(f"Successfully created subtitled video: {output_video_path}")
    except subprocess.CalledProcessError as e:
        print("Error during ffmpeg processing:")
        print(e.stderr)
